# Predição de Casos de COVID com AutoML

In [ ]:
from PIL import Image
pil_im = Image.open('../input/logocanal/LOGO PNG.png')
pil_im

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        pass
# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Lendo os dados

In [ ]:
import pandas as pd
from statistics import mean 
df=pd.read_csv('/kaggle/input/corona-virus-report/covid_19_clean_complete.csv')

# Definindo uma função para ler os Dados de X e Y

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from numpy import array


def country_data(country):

    tscv = TimeSeriesSplit()

    df_country=df[df['Country/Region']==country]
    df_country_grouped=df_country.groupby(['Date']).sum()


    scores=pd.DataFrame(columns=['Model','R2'])

    X_train_list=[]
    X_test_list=[]
    y_train_list=[]
    y_test_list=[]


    series=df_country_grouped['Active']
    # split a univariate sequence into samples
    def split_sequence(sequence, n_steps):
        X, y = list(), list()
        for i in range(len(sequence)):
            # find the end of this pattern
            end_ix = i + n_steps
            # check if we are beyond the sequence
            if end_ix > len(sequence)-1:
                break
            # gather input and output parts of the pattern
            seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
            X.append(seq_x)
            y.append(seq_y)
        return array(X), array(y)

    # choose a number of time steps
    n_steps = 25
    # split into samples
    X, y = split_sequence(series, n_steps)

    from sklearn.preprocessing import MinMaxScaler

    X=MinMaxScaler().fit_transform(X,y)
    
    return X,y


    

# Separando os dados de X e Y

In [ ]:
X,y=country_data('Italy')

# Instalando as ferramentas necessárias

In [ ]:
!pip install requests
!pip install tabulate
!pip install "colorama>=0.3.8"
!pip install future

In [ ]:
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

## Iniciando Servidor de H2O

In [ ]:
import h2o
h2o.init()

## Treinando e escolhendo melhores Modelos

In [ ]:
X=pd.DataFrame(X)
X['target']=y
data = h2o.H2OFrame(X)

splits = data.split_frame(ratios=[0.7, 0.15], seed=1)  

train = splits[0]
valid = splits[1]
test = splits[2]

y_ = 'target'
x = list(data.columns)

x.remove(y_)  
from h2o.automl import H2OAutoML

aml = H2OAutoML(max_models=5, seed=1)
aml.train(x=x, y=y_, training_frame=train)


# Listando melhores modelos

In [ ]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)


# Apresentando o score e o resultado alcançado

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
#X_test = np.array(X_test).reshape((X_test.shape[0], X_test.shape[1],1))
#     # fit model

predictions=aml.predict(test)
count=1



ypred=list(predictions.as_data_frame()['predict'])
print(ypred)
y_test=list(test.as_data_frame()['target'])
print(y_test)
error = mean_squared_error(y_test, ypred)

title='H2O Italy :'+str(1)
count+=1
fig = plt.figure()
fig.suptitle(title, fontsize=12)
plt.plot(y_test)
plt.plot(ypred, color='red')
plt.show()
r2_score_ = r2_score(y_test, ypred)
mse_score=mean_squared_error(y_test, ypred)
mae_score=mean_absolute_error(y_test, ypred)


print('R2 Score:',r2_score_)
print('MSE Score:',mse_score)
print('MAE Score:',mae_score)



